Import of libraries

In [9]:
from __future__ import print_function
# We'll need numpy for some mathematical operations
import numpy as np

# matplotlib for displaying the output
import matplotlib.pyplot as plt
%matplotlib inline

# and IPython.display for audio output
import IPython.display

# Librosa for audio
import librosa
# And the display module for visualization
import librosa.display

import parselmouth

from parselmouth.praat import call
from sklearn.decomposition import PCA
import statistics
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

Descriptive statistics for MELD dataset

In [11]:
import pandas as pd
import os
parent_dir = "meld" #folder which contain MELD audios
def prepare_datadf(parent_dir): # a function whose parameter is the audio folder
    df = pd.DataFrame(columns = ['audio']) #dataframe columns
    
    for  fichier_audio in os.listdir(parent_dir):
      # for each element in the audio folder

     
        
      if fichier_audio.endswith('outNoise.wav'):
                    
        df = df.append(pd.DataFrame({'audio':[fichier_audio]}), 
                           ignore_index=True) # here at df defined, with the columns we add the values:
                                            #the audio column will take the audios_path, 
                                            #and the emotion column will take the corresponding emotion, ie the name of the folder
    #Shuffling for randomness
    df = df.sample(frac=1.0).reset_index(drop=True)
    return df
datadf = prepare_datadf(parent_dir) #function call
display(datadf.head()) #dataframe display

,audio
0,dia106_utt3_norm_outNoise.wav
1,dia99_utt1_norm_outNoise.wav
2,dia107_utt3_norm_outNoise.wav
3,dia23_utt4_norm_outNoise.wav
4,dia40_utt0_norm_outNoise.wav


read of CSV file : file that annotates the audios of the dev folder


In [12]:
datacsv = pd.read_csv("dev_sent_emo.csv", encoding="utf8")
display(datacsv.head())
print(len(datacsv))

,Sr No.,Utterance,Speaker,Emotion,Sentiment,Dialogue_ID,Utterance_ID,Season,Episode,StartTime,EndTime
0,1,"Oh my God, hes lost it. Hes totally lost it.",Phoebe,sadness,negative,0,0,4,7,"00:20:57,256","00:21:00,049"
1,2,What?,Monica,surprise,negative,0,1,4,7,"00:21:01,927","00:21:03,261"
2,3,"Or! Or, we could go to the bank, close our acc...",Ross,neutral,neutral,1,0,4,4,"00:12:24,660","00:12:30,915"
3,4,Youre a genius!,Chandler,joy,positive,1,1,4,4,"00:12:32,334","00:12:33,960"
4,5,"Aww, man, now we wont be bank buddies!",Joey,sadness,negative,1,2,4,4,"00:12:34,211","00:12:37,505"


1109


Labelling : association of each audio to his emotion

In [14]:
import re
df = pd.DataFrame(columns = ['audio','dialogue', 'utterance', 'emotion']) #dataframe columns
array = datadf.values
audios = array[:,0]

    
for  fichier_audio in audios:
  sep = re.split("\_", fichier_audio)
  dia = sep[0][3:]
  dia = int(dia)
  utt = sep[1][3:]
  utt = int(utt)
  #print(dia, utt)
  for index, row in datacsv.iterrows():
    list = [row['Dialogue_ID'], row['Utterance_ID'], row['Emotion']]
    #print(list)


    if dia == list[0] and utt == list[1]:
      df = df.append(pd.DataFrame({'audio':[fichier_audio], 'dialogue' : [list[0]], 'utterance' : [list[1]], 'emotion' : [list[2]]}), 
                          ignore_index=True) # here at df defined, with the columns we add the values:
                                            #the audio column will take the audios_path, 
                                            #and the emotion column will take the corresponding emotion, ie the name of the folder

display(df.head())

,audio,dialogue,utterance,emotion
0,dia106_utt3_norm_outNoise.wav,106,3,neutral
1,dia99_utt1_norm_outNoise.wav,99,1,neutral
2,dia107_utt3_norm_outNoise.wav,107,3,surprise
3,dia23_utt4_norm_outNoise.wav,23,4,sadness
4,dia40_utt0_norm_outNoise.wav,40,0,surprise


In [26]:
folder = parent_dir
def tableau(folder):
    daf = pd.DataFrame(columns = ['audio', 'pitch', 'duration', 'energy', 'zcr', 'mfccs', 'lpc', 'HNR',
                                  'localJitter', 'localabsoluteJitter',  'localShimmer', 'localdbShimmer',
                                  'f1_mean', 'f2_mean', 'f3_mean', 'f4_mean', 'rapJitter', 'ppq5Jitter', 'ddpJitter',
                                  'apq3Shimmer', 'aqpq5Shimmer', 'apq11Shimmer', 'ddaShimmer'])
    for audio_file in array[:,0]:
        if audio_file.endswith('.wav'):
            
            #features = extract_features(folder+'/'+audio_file)
            #all_features.append(features)
            y,sr = librosa.load(folder+'/'+ audio_file)
            
            zcr = librosa.feature.zero_crossing_rate(y)
            mean_zcr = np.mean(zcr)
            
            ############################coefficients cepstraux de merl##################################
            mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
            
            mean_mfccs = np.mean(mfccs)
            
            duration = librosa.get_duration(y=y, sr=sr)
            energy = np.sum(y ** 2) / np.float64(len(y))
            
            lpc = librosa.core.lpc(y,16)
            mean_lpc = np.mean(lpc)
            
            sound = parselmouth.Sound(folder+'/'+ audio_file)
            pitch = call(sound, "To Pitch", 0.0, 75, 600)
            mean_pitch = call(pitch, "Get mean", 0, 0, "Hertz")
            
            
            harmonicity = call(sound, "To Harmonicity (cc)", 0.01, 75, 0.1, 1.0)
            HNR = call(harmonicity, "Get mean", 0, 0)
            
            pointProcess = call(sound, "To PointProcess (periodic, cc)", 75, 500)
            localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3)
            localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3)

            localShimmer =  call([sound, pointProcess], "Get shimmer (local)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
            localdbShimmer = call([sound, pointProcess], "Get shimmer (local_dB)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
                
            formants = call(sound, "To Formant (burg)", 0.0, 5, 5500, 0.025, 100)
            numPoints = call(pointProcess, "Get number of points")

            f1_list = []
            f2_list = []
            f3_list = []
            f4_list = []
    
            # Measure formants only at glottal pulses
            for point in range(0, numPoints):
                point += 1
                t = call(pointProcess, "Get time from index", point)
                f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
                f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
                f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
                f4 = call(formants, "Get value at time", 4, t, 'Hertz', 'Linear')
                f1_list.append(f1)
                f2_list.append(f2)
                f3_list.append(f3)
                f4_list.append(f4)
    
            f1_list = [f1 for f1 in f1_list if str(f1) != 'nan']
            f2_list = [f2 for f2 in f2_list if str(f2) != 'nan']
            f3_list = [f3 for f3 in f3_list if str(f3) != 'nan']
            f4_list = [f4 for f4 in f4_list if str(f4) != 'nan']

            f1_mean = statistics.mean(f1_list)
            f2_mean = statistics.mean(f2_list)
            f3_mean = statistics.mean(f3_list)
            f4_mean = statistics.mean(f4_list)

            rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3)
            ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3)
            ddpJitter = call(pointProcess, "Get jitter (ddp)", 0, 0, 0.0001, 0.02, 1.3)   
            
            apq3Shimmer = call([sound, pointProcess], "Get shimmer (apq3)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
            aqpq5Shimmer = call([sound, pointProcess], "Get shimmer (apq5)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
            apq11Shimmer =  call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
            ddaShimmer = call([sound, pointProcess], "Get shimmer (dda)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    
            
            
        
        daf = daf.append(pd.DataFrame({'audio':[audio_file], 'pitch':[ mean_pitch], 
                                       'duration':[duration], 'energy' : [energy], 'zcr':[mean_zcr], 'mfccs':[mean_mfccs]
                                      , 'lpc' : [mean_lpc], 'HNR':[HNR], 
                                       'localJitter':[localJitter], 
                                       'localabsoluteJitter':[localabsoluteJitter],
                                       ' localShimmer':[ localShimmer], 
                                       'localdbShimmer ':[localdbShimmer], 'f1_mean': [f1_mean],
                                       'f2_mean': [f2_mean],  'f3_mean': [f3_mean],  'f4_mean': [f4_mean], 
                                       'rapJitter': [rapJitter], 'ppq5Jitter': [ppq5Jitter], 'ddpJitter': [ddpJitter],
                                      'apq3Shimmer': [apq3Shimmer], 'aqpq5Shimmer': [aqpq5Shimmer], 
                                       'apq11Shimmer': [apq11Shimmer], 'ddaShimmer': [ddaShimmer]}), 
                           ignore_index=True)
            
    daf = daf.sample(frac=1.0).reset_index(drop=True)
    return daf
datatab = tableau(folder) #function call
display(datatab.head()) #dataframe display

,audio,pitch,duration,energy,zcr,mfccs,lpc,HNR,localJitter,localabsoluteJitter,localShimmer,localdbShimmer,f1_mean,f2_mean,f3_mean,f4_mean,rapJitter,ppq5Jitter,ddpJitter,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,localShimmer,localdbShimmer
0,dia72_utt5_norm_outNoise.wav,455.898534,1.02,0.012234,0.337336,-23.055824,0.074842,5.026421,0.017686,0.000055,NaN,NaN,1049.776420,1928.794936,3033.681170,4058.455147,0.008159,0.011272,0.024476,0.066206,0.094821,0.126266,0.198619,0.142557,1.337437
1,dia21_utt0_norm_outNoise.wav,344.918044,1.74,0.011669,0.182695,-20.833315,0.023122,10.004505,0.026188,0.000079,NaN,NaN,881.134168,1512.870928,3119.682911,3751.165591,0.012925,0.013795,0.038774,0.062971,0.107029,0.225887,0.188912,0.164103,1.515416
2,dia5_utt11_norm_outNoise.wav,421.231176,0.50,0.004375,0.170676,-14.289187,0.030666,5.324701,0.029990,0.000085,NaN,NaN,986.855285,1857.004267,2875.811889,3864.246822,0.014819,0.016236,0.044456,0.061450,0.086969,0.205059,0.184349,0.174015,1.472696
3,dia69_utt11_norm_outNoise.wav,404.334443,0.86,0.013319,0.248021,-25.211637,0.015172,7.820126,0.020039,0.000050,NaN,NaN,744.008916,1452.201867,2992.907939,3919.111601,0.010090,0.010991,0.030269,0.053771,0.069798,0.116052,0.161312,0.129965,1.202194
4,dia46_utt9_norm_outNoise.wav,465.080234,1.42,0.006221,0.225594,-29.128784,0.054821,5.639640,0.030029,0.000079,NaN,NaN,1035.412613,1747.020182,3368.148593,4019.110578,0.015814,0.019564,0.047441,0.060364,0.101825,0.205273,0.181091,0.167346,1.570320


Statistics

In [27]:

datatab.describe()

,pitch,duration,energy,zcr,mfccs,lpc,HNR,localJitter,localabsoluteJitter,f1_mean,f2_mean,f3_mean,f4_mean,rapJitter,ppq5Jitter,ddpJitter,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,localShimmer,localdbShimmer
count,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000,1047.000000
mean,387.873467,1.612226,0.012596,0.198588,-22.446096,0.029637,7.984106,0.025682,0.000077,901.023240,1725.847963,2905.507614,3932.833132,0.012190,0.014324,0.036571,0.058339,0.089157,0.159943,0.175018,0.145710,1.360941
std,69.543246,1.245582,0.007681,0.047724,4.479308,0.024248,3.014244,0.008903,0.000031,155.532479,172.970863,192.822600,186.072685,0.005291,0.006395,0.015872,0.014929,0.022719,0.057138,0.044787,0.027382,0.193062
min,180.269359,0.140000,0.000447,0.075703,-35.197449,0.001423,-1.101477,0.004471,0.000015,504.255220,1098.284918,2172.218671,3347.257885,0.001468,0.002328,0.004403,0.017515,0.026269,0.006690,0.052544,0.061431,0.629538
25%,346.308644,0.820000,0.007160,0.165968,-25.349572,0.015178,6.014373,0.019706,0.000056,798.571011,1613.879782,2792.993430,3813.821610,0.008624,0.010232,0.025873,0.048222,0.074783,0.125831,0.144667,0.127329,1.244407
50%,401.975242,1.280000,0.011227,0.195645,-22.824852,0.023581,8.130862,0.024311,0.000072,880.379839,1720.059261,2909.132166,3927.423518,0.011430,0.013186,0.034289,0.057215,0.086276,0.152628,0.171645,0.144044,1.358448
75%,437.817525,1.990000,0.016061,0.227469,-19.738965,0.036559,9.901349,0.030074,0.000089,986.148032,1832.934058,3024.081153,4050.334305,0.014545,0.016987,0.043635,0.067310,0.101581,0.185253,0.201929,0.163824,1.483623
max,551.276140,14.250023,0.075034,0.468974,-1.702829,0.273830,22.018230,0.091614,0.000380,1636.308384,2477.022068,3571.564297,4604.028013,0.057952,0.078108,0.173857,0.136482,0.283521,0.830141,0.409447,0.309372,2.100474
